In [1]:
import pandas as pd
import numpy as np

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Practice').getOrCreate()

In [3]:
df=spark.read.option('header','true').csv('C://Users//Durgesh//Documents//Space+Missions//space_missions.csv',inferSchema=True)

In [4]:
df.show()

+------------+--------------------+---------+--------+------------------+--------------+------------+-----+-------------+
|     Company|            Location|     Date|    Time|            Rocket|       Mission|RocketStatus|Price|MissionStatus|
+------------+--------------------+---------+--------+------------------+--------------+------------+-----+-------------+
|   RVSN USSR|Site 1/5, Baikonu...|10/4/1957|19:28:00|    Sputnik 8K71PS|     Sputnik-1|     Retired| null|      Success|
|   RVSN USSR|Site 1/5, Baikonu...|11/3/1957| 2:30:00|    Sputnik 8K71PS|     Sputnik-2|     Retired| null|      Success|
|     US Navy|LC-18A, Cape Cana...|12/6/1957|16:44:00|          Vanguard|  Vanguard TV3|     Retired| null|      Failure|
|        AMBA|LC-26A, Cape Cana...| 2/1/1958| 3:48:00|            Juno I|    Explorer 1|     Retired| null|      Success|
|     US Navy|LC-18A, Cape Cana...| 2/5/1958| 7:33:00|          Vanguard|Vanguard TV3BU|     Retired| null|      Failure|
|        AMBA|LC-26A, Ca

In [5]:
df.describe()

DataFrame[summary: string, Company: string, Location: string, Date: string, Time: string, Rocket: string, Mission: string, RocketStatus: string, Price: string, MissionStatus: string]

In [6]:
from pyspark.sql.functions import isnan, when, count, col
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+--------+----+----+------+-------+------------+-----+-------------+
|Company|Location|Date|Time|Rocket|Mission|RocketStatus|Price|MissionStatus|
+-------+--------+----+----+------+-------+------------+-----+-------------+
|      0|       0|   0| 127|     0|      0|           0| 3365|            0|
+-------+--------+----+----+------+-------+------------+-----+-------------+



In [7]:
df.count()

4630

- In the following analysis we found that price value is null only for RocketStatus having Status as Retired

In [8]:
df.select(col("RocketStatus"),col("Price")).filter(df.RocketStatus == 'Retired').show()

+------------+-----+
|RocketStatus|Price|
+------------+-----+
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
|     Retired| null|
+------------+-----+
only showing top 20 rows



- As quantity of null value is much higher so it will not contribute more to analysis so we will remove column Price

In [9]:
df=df.drop(col("Price"))

In [10]:
df.show()

+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+
|     Company|            Location|     Date|    Time|            Rocket|       Mission|RocketStatus|MissionStatus|
+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+
|   RVSN USSR|Site 1/5, Baikonu...|10/4/1957|19:28:00|    Sputnik 8K71PS|     Sputnik-1|     Retired|      Success|
|   RVSN USSR|Site 1/5, Baikonu...|11/3/1957| 2:30:00|    Sputnik 8K71PS|     Sputnik-2|     Retired|      Success|
|     US Navy|LC-18A, Cape Cana...|12/6/1957|16:44:00|          Vanguard|  Vanguard TV3|     Retired|      Failure|
|        AMBA|LC-26A, Cape Cana...| 2/1/1958| 3:48:00|            Juno I|    Explorer 1|     Retired|      Success|
|     US Navy|LC-18A, Cape Cana...| 2/5/1958| 7:33:00|          Vanguard|Vanguard TV3BU|     Retired|      Failure|
|        AMBA|LC-26A, Cape Cana...| 3/5/1958|18:27:00|            Juno I

- Now we will take action on null values of Time Column

In [11]:
df.select(col("Time"),col("Mission")).filter(df.Time.isNull()).show()

+----+--------------------+
|Time|             Mission|
+----+--------------------+
|null|        Pilot-1 (D1)|
|null|        Pilot-2 (D2)|
|null|        Pilot-3 (D3)|
|null|        Pilot-4 (R1)|
|null|        Pilot-5 (R2)|
|null|         Pilot-6 (R3|
|null|   E-3 n†­3 (Luna-3)|
|null|DSAP-1 F1 (Progra...|
|null|DSAP-1 F4 (Progra...|
|null|            DS-MT #1|
|null|             Midas 8|
|null|      Elektron 3 & 4|
|null|    KH-7 Gambit 4012|
|null|   Strela-1 #4, 5, 6|
|null|             DS-2 #2|
|null|           DS-K40 #1|
|null|           DS-K40 #2|
|null|               Osumi|
|null|               Osumi|
|null|               Osumi|
+----+--------------------+
only showing top 20 rows



- The time values are null and we are not able to determine exact reason why value for Time column is null . Thus, we will try to put arbitrary value which does not exist in Time column
- I have considered arbitrary value as 1:13:00 because 1:13:00 value is not present Time column

In [12]:
df=df.na.fill({"Time":"1:13:00"})

In [55]:
dataframe=df.withColumn("Year",split(df.Date,'/').getItem(2))
dataframe.show()

+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+----+
|     Company|            Location|     Date|    Time|            Rocket|       Mission|RocketStatus|MissionStatus|Year|
+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+----+
|   RVSN USSR|Site 1/5, Baikonu...|10/4/1957|19:28:00|    Sputnik 8K71PS|     Sputnik-1|     Retired|      Success|1957|
|   RVSN USSR|Site 1/5, Baikonu...|11/3/1957| 2:30:00|    Sputnik 8K71PS|     Sputnik-2|     Retired|      Success|1957|
|     US Navy|LC-18A, Cape Cana...|12/6/1957|16:44:00|          Vanguard|  Vanguard TV3|     Retired|      Failure|1957|
|        AMBA|LC-26A, Cape Cana...| 2/1/1958| 3:48:00|            Juno I|    Explorer 1|     Retired|      Success|1958|
|     US Navy|LC-18A, Cape Cana...| 2/5/1958| 7:33:00|          Vanguard|Vanguard TV3BU|     Retired|      Failure|1958|
|        AMBA|LC-26A, Cape Cana.

In [69]:
from pyspark.sql.functions import *
dataframe.withColumn("Countries",split(df.Location,',').getItem(2)).show()


+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+----+-----------+
|     Company|            Location|     Date|    Time|            Rocket|       Mission|RocketStatus|MissionStatus|Year|  Countries|
+------------+--------------------+---------+--------+------------------+--------------+------------+-------------+----+-----------+
|   RVSN USSR|Site 1/5, Baikonu...|10/4/1957|19:28:00|    Sputnik 8K71PS|     Sputnik-1|     Retired|      Success|1957| Kazakhstan|
|   RVSN USSR|Site 1/5, Baikonu...|11/3/1957| 2:30:00|    Sputnik 8K71PS|     Sputnik-2|     Retired|      Success|1957| Kazakhstan|
|     US Navy|LC-18A, Cape Cana...|12/6/1957|16:44:00|          Vanguard|  Vanguard TV3|     Retired|      Failure|1957|    Florida|
|        AMBA|LC-26A, Cape Cana...| 2/1/1958| 3:48:00|            Juno I|    Explorer 1|     Retired|      Success|1958|    Florida|
|     US Navy|LC-18A, Cape Cana...| 2/5/1958| 7:33:00|          Vangu